In [1]:
import torch
from torch import nn
import numpy as np
import pandas
from torch.utils.data import Dataset, DataLoader
import time

pandas.set_option('precision', 20)

In [2]:
def get_features(dataframe:pandas.DataFrame)->torch.Tensor:
        return dataframe[["Theta", "Pitch", "Duty Cycle", "Fill Factor", "Lambda", "Mode"]].values

In [5]:
dataset = pandas.read_csv('DATA_FILES/dataset_killed_low_vals.csv')
dataset

,Theta,Pitch,Duty Cycle,Fill Factor,Mode,Lambda,0
0,5.0,0.00000050,0.40000000000000002220,0.20000000000000001110,0,0.00000170,0.00000000000000000000
1,5.0,0.00000050,0.40000000000000002220,0.20000000000000001110,0,0.00000170,0.00000000000000000000
2,5.0,0.00000050,0.40000000000000002220,0.20000000000000001110,0,0.00000169,0.00000000000000000000
3,5.0,0.00000050,0.40000000000000002220,0.20000000000000001110,0,0.00000169,0.00000000000000000000
4,5.0,0.00000050,0.40000000000000002220,0.20000000000000001110,0,0.00000169,0.00000000000000000000
...,...,...,...,...,...,...,...
602195,11.0,0.00000151,0.44444399999999995021,0.55555600000000004979,0,0.00000131,0.02360320000000000126
602196,11.0,0.00000151,0.44444399999999995021,0.55555600000000004979,0,0.00000130,0.02386749999999999983
602197,11.0,0.00000151,0.44444399999999995021,0.55555600000000004979,0,0.00000130,0.02406900000000000012
602198,11.0,0.00000151,0.44444399999999995021,0.55555600000000004979,0,0.00000130,0.02422159999999999938


In [6]:
columns_to_norm = ['Theta', 'Pitch', 'Duty Cycle', 'Fill Factor', 'Lambda', 'Mode']
dataset[columns_to_norm] = dataset[columns_to_norm].apply(lambda x: (x-x.min()) / (x.max() - x.min()))
dataset

,Theta,Pitch,Duty Cycle,Fill Factor,Mode,Lambda,0
0,0.0000000000000000000,0.0,0.00000000000000000000,0.00000000000000000000,0.0,1.00000000000000000000,0.00000000000000000000
1,0.0000000000000000000,0.0,0.00000000000000000000,0.00000000000000000000,0.0,1.00000000000000000000,0.00000000000000000000
2,0.0000000000000000000,0.0,0.00000000000000000000,0.00000000000000000000,0.0,0.97499999999999975575,0.00000000000000000000
3,0.0000000000000000000,0.0,0.00000000000000000000,0.00000000000000000000,0.0,0.97499999999999975575,0.00000000000000000000
4,0.0000000000000000000,0.0,0.00000000000000000000,0.00000000000000000000,0.0,0.97499999999999975575,0.00000000000000000000
...,...,...,...,...,...,...,...
602195,0.4000000000000000222,1.0,0.11110999999999982002,0.88889000000000017998,0.0,0.02499999999999976200,0.02360320000000000126
602196,0.4000000000000000222,1.0,0.11110999999999982002,0.88889000000000017998,0.0,0.00000000000000000000,0.02386749999999999983
602197,0.4000000000000000222,1.0,0.11110999999999982002,0.88889000000000017998,0.0,0.00000000000000000000,0.02406900000000000012
602198,0.4000000000000000222,1.0,0.11110999999999982002,0.88889000000000017998,0.0,0.00000000000000000000,0.02422159999999999938


In [7]:
dataset.to_csv('DATA_FILES/dataset_killed_low_vals_min_max_normalized.csv')

In [8]:
training_set = dataset.sample(frac = 0.85)
training_set

,Theta,Pitch,Duty Cycle,Fill Factor,Mode,Lambda,0
584362,0.3697000000000000286,1.00000000000000000000,0.11110999999999982002,0.88889000000000017998,1.0,0.14999999999999963363,0.00000000000000000000
168925,0.2000000000000000111,0.99009900990099020124,0.79999999999999982236,0.39999999999999996669,0.0,0.29999999999999976685,0.00030406800000000000
534137,0.2195286666666666775,0.38217821782178212908,1.00000000000000000000,1.00000000000000000000,0.0,0.24999999999999972244,0.00993110999999999994
281796,0.5999999999999999778,0.19801980198019800139,0.59999999999999997780,0.39999999999999996669,0.0,0.00000000000000000000,0.00452254000000000024
342429,0.5999999999999999778,0.99009900990099020124,0.79999999999999982236,0.80000000000000004441,0.0,0.82500000000000006661,0.01616520000000000110
...,...,...,...,...,...,...,...
67267,0.0000000000000000000,0.79207920792079211658,0.79999999999999982236,0.00000000000000000000,0.0,0.60000000000000008882,0.00196924000000000014
123620,0.2000000000000000111,0.39603960396039600278,0.59999999999999997780,0.59999999999999997780,0.0,0.87500000000000011102,0.01855949999999999961
229611,0.4000000000000000222,0.59405940594059414295,1.00000000000000000000,0.80000000000000004441,0.0,0.92500000000000015543,0.00054426900000000000
268563,0.5999999999999999778,0.00000000000000000000,0.59999999999999997780,1.00000000000000000000,0.0,0.14999999999999963363,0.00000000000000000000


In [9]:
training_set.to_csv('DATA_FILES/training_set_killed_low_vals.csv')

In [10]:
testing_set = dataset.drop(training_set.index)
testing_set

,Theta,Pitch,Duty Cycle,Fill Factor,Mode,Lambda,0
3,0.0000000000000000000,0.0,0.00000000000000000000,0.00000000000000000000,0.0,0.97499999999999975575,0.00000000000000000000
8,0.0000000000000000000,0.0,0.00000000000000000000,0.00000000000000000000,0.0,0.94999999999999995559,0.00000000000000000000
34,0.0000000000000000000,0.0,0.00000000000000000000,0.00000000000000000000,0.0,0.79999999999999982236,0.00000000000000000000
37,0.0000000000000000000,0.0,0.00000000000000000000,0.00000000000000000000,0.0,0.77500000000000002220,0.00000000000000000000
39,0.0000000000000000000,0.0,0.00000000000000000000,0.00000000000000000000,0.0,0.74999999999999977796,0.00000000000000000000
...,...,...,...,...,...,...,...
602178,0.4000000000000000222,1.0,0.11110999999999982002,0.88889000000000017998,0.0,0.07499999999999981681,0.00909586000000000075
602184,0.4000000000000000222,1.0,0.11110999999999982002,0.88889000000000017998,0.0,0.05000000000000005135,0.01538750000000000014
602188,0.4000000000000000222,1.0,0.11110999999999982002,0.88889000000000017998,0.0,0.05000000000000005135,0.01941959999999999867
602197,0.4000000000000000222,1.0,0.11110999999999982002,0.88889000000000017998,0.0,0.00000000000000000000,0.02406900000000000012


In [12]:
testing_set.to_csv('DATA_FILES/testing_set_killed_low_vals.csv')